## Check List 4 - PlanningPipeline
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4.1 PlanningScene**  
  - create_binder
    - actor: PlacePlane, Gripper2Tool, SweepTool
    - handle: Grasp2Point, PlacePoint, SweepPoint
  - create_subject
    - subject: CustomObject, SweepLineTask
  - initialize_state
* **4.2 MotionPlanner**  
  - MoveitPlanner
    - pick, place, sweep
    - Interpolated path generation by incremental IK
* **4.3 TaskPlanner**  
  - RRT
    - pick & place & sweep
      - single process
      - multi process
  - PDDLStream
    - pick & place

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=np.random.rand(3)*0, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
#                            rpy=np.random.rand(3)*np.pi/16, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=True)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepFramer

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

In [7]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name="indy0_tcp", 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name="indy0_tcp",
                              dims=(0.04,0.02,0.1), center=(0,0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name="indy0_tcp", 
                              dims=(0.04,0.02,0.1), center=(0,-0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

In [8]:
# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepFramer, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [9]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepFrame
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [10]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [11]:
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepFrame("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepFrame("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [12]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

('floor', 0)


## 4.2 MotionPlanner

##### MoveitPlanner
* *get_available_binding_dict()* gets available binding states for each subject in a dictionary
* *sample_leaf_state samples()* target state with given available_binding_dict and target node
* *rebind_all()* updates binding state and returns the resultant state
* The motions tested in this section are:
  - pick: move the object to "gripper"
  - place: move the object to "goal"
  - sweep: 
    1) approach to waypoint 1
    2) sweep to waypoint 2
    3) return to home pose

In [13]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType, PlannerConfig

gtimer = GlobalTimer.instance()
gtimer.reset()

In [14]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, motion_filters=[GraspChecker(pscene)])

In [15]:
# pick motion - move the object to "gripper"
from_state = initial_state.copy(pscene)
to_node = ("gripper", 0)
for _ in range(10):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    with gtimer.block("plan"):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    pick_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

In [16]:
# place motion - move the object to "goal"
from_state = pick_state.copy(pscene)
to_node = ("goal", 0)
for _ in range(20):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    print("Sometimes this placement motion is inherently impossible depending on pick parameter.")
    print("Then, please re-try from the beginning to change the configuration")
    raise(RuntimeError("Motion plan failure"))

In [17]:
# sweep motion 1 - reach the first waypoint
from_state = initial_state.copy(pscene)
from_state.Q = crob.home_pose
to_node = ("floor", 1)
for _ in range(3):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, timeout=1)
    if success:
        if np.abs(LastQ[0])<np.pi/2: # else, it will fail in sweep motion
            break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    sweep1_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

In [18]:
# sweep motion 2 - sweep "floor" to the second waypoint. Constrained motion planning takes longer time
from_state = sweep1_state.copy(pscene)
to_node = ("floor", 2)
for _ in range(5):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, timeout=5)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    sweep2_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    print("Sometimes this constrained motion is inherently impossible.")
    print("Then, please re-try from the beginning to change the configuration")
    raise(RuntimeError("Motion plan failure"))

* Interpolated path generation by incremental IK
  - To generate task-space Interpolated path, set MoveitPlanner.incremental_constraint_motion = True
  - If the changing subject returns non-zero constraints with make_constraints, this task-space interpolation will be applied.
  - For simple linear moion, this is fast way to generate constrained motion.
  - This only does linear interpolation and does not observe the constraints.

In [19]:
# Set incremental path generation flag
mplan.incremental_constraint_motion = True

from_state = sweep1_state.copy(pscene)
to_node = ("floor", 2)
for _ in range(5):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, timeout=5)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    sweep2_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    print("Sometimes this constrained motion is inherently impossible.")
    print("Then, please re-try from the beginning to change the configuration")
    raise(RuntimeError("Motion plan failure"))

## 4.3 TaskPlanner

##### create PlanningPipeline

In [20]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene))

In [21]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


##### RRT

In [22]:
from pkg.planning.task.rrt import TaskRRT
ppline.set_task_planner(TaskRRT(pscene))

   Use a production WSGI server instead.
 * Debug mode: off


- pick & place & sweep (single process)
  - this can fail due to short timeout

In [23]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=10, max_solution_count=1, 
              timeout=0.5, timeout_constrained=2, multiprocess=False)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.01)

try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (0.06/10.0 s, steps/err: 10(50.0328540802 ms)/0.00199918969503)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 1 - ('floor', 1)->('floor', 2)
end
constrained motion tried: True
result: 1 - ('floor', 1)->('floor', 2) = success
branching: 1->2 (0.82/10.0 s, steps/err: 41(203.440189362 ms)/0.000102036004388)
try: 2 - ('floor', 2)->('gripper', 2)
try transition motion
transition motion tried: True
result: 2 - ('floor', 2)->('gripper', 2) = success
branching: 2->3 (0.88/10.0 s, steps/err: 39(60.1980686188 ms)/0.00193757995714)
try: 3 - ('gripper', 2)->('goal', 2)
try transition motion
transition motion tried: True
Goal reached
result: 3 - ('gripper', 2)->('goal', 2) = success
branching: 3->4 (0.92/10.0 s, steps/err: 51(36.777

- pick & place & sweep (multi process)

In [24]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=10, max_solution_count=1, 
              timeout=0.5, timeout_constrained=2, multiprocess=True)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.01)

Use 18/36 agents
try: 0 - ('floor', 0)->('gripper', 0)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
result: 0 - ('floor', 0)->('floor', 1) = success
transition motion tried: True
branching: 0->1 (0.17/10.0 s, steps/err: 10(61.2399578094 ms)/0.00124472049848)
try transition motion
transition motion tried: True
transition motion tried: True
try: 1 - ('floor', 1)->('floor', 2)
result: 0 - ('floor', 0)->('floor', 1) = success
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->2 (0.24/10.0 s, steps/err: 10(88.0148410797 ms)/0.

try: 4 - ('gripper', 0)->('gripper', 1)
try transition motion
try transition motion
try: 6 - ('gripper', 0)->('goal', 0)
end
try transition motion
transition motion tried: True
constrained motion tried: True
transition motion tried: True
transition motion tried: True
result: 8 - ('floor', 1)->('floor', 2) = success
transition motion tried: False
result: 7 - ('floor', 0)->('floor', 1) = success
Motion Plan Failure
result: 0 - ('floor', 0)->('gripper', 0) = fail
branching: 8->21 (0.89/10.0 s, steps/err: 41(445.657014847 ms)/0.000101851902432)
try: 21 - ('floor', 2)->('gripper', 2)
result: 4 - ('gripper', 0)->('gripper', 1) = success
try transition motion
result: 7 - ('floor', 0)->('floor', 1) = success
branching: 7->20 (0.88/10.0 s, steps/err: 43(376.447916031 ms)/0.00124472049848)
branching: 4->22 (0.91/10.0 s, steps/err: 24(83.5320949554 ms)/0.00185717369631)
try: 10 - ('gripper', 0)->('floor', 0)
try: 5 - ('gripper', 0)->('goal', 0)
try: 13 - ('gripper', 1)->('gripper', 2)
try transit

transition motion tried: False
try: 38 - ('goal', 1)->('goal', 2)
Motion Plan Failure
result: 11 - ('floor', 2)->('gripper', 2) = fail
Goal reached
try: 5 - ('gripper', 0)->('goal', 0)
try transition motion
transition motion tried: True
end
constrained motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->40 (1.26/10.0 s, steps/err: 35(198.46200943 ms)/0.00137568124381)
Goal reached
result: 13 - ('gripper', 1)->('gripper', 2) = success
try: 14 - ('goal', 0)->('goal', 1)
branching: 13->42 (1.27/10.0 s, steps/err: 41(324.676990509 ms)/0.000101777368572)
try transition motion
try: 42 - ('gripper', 2)->('goal', 2)
try transition motion
transition motion tried: True
result: 7 - ('floor', 0)->('gripper', 0) = success
Goal reached
branching: 7->43 (1.3/10.0 s, steps/err: 63(209.076881409 ms)/0.00182120945925)
try: 26 - ('goal', 0)->('gripper', 0)
try transition motion
transition motion tried: True
transition motion tried: False
transition motion tried: True
Motio

branching: 32->58 (1.54/10.0 s, steps/err: 4(83.9269161224 ms)/0.00142164381843)
======================= terminated 15: Stop called from other agent  (1.5/10.0) ===============================
======================= terminated 5: Stop called from other agent  (1.6/10.0) ===============================
======================= terminated 0: Stop called from other agent  (1.6/10.0) ===============================
transition motion tried: True
Goal reached
result: 30 - ('gripper', 2)->('goal', 2) = success
branching: 30->59 (1.58/10.0 s, steps/err: 64(105.594873428 ms)/0.0014815460589)
++ adding return motion to acquired answer ++
end
constrained motion tried: True
Goal reached
result: 45 - ('goal', 1)->('goal', 2) = success
branching: 45->60 (1.62/10.0 s, steps/err: 41(249.078035355 ms)/0.000101819301892)
++ adding return motion to acquired answer ++
======================= terminated 11: required answers acquired  (1.6/10.0) ===============================
Goal reached
Goal reached
join

##### PDDLStream
* pick & place
* currently only *single robot* is supported
* only colliding geometries are transfered to PDDLStream(pybullet). make sure collision=True for all subjects and actors.

In [25]:
pscene.set_object_state(initial_state)
## Remove sweep - currently not supported with PDDLStream
pscene.remove_subject("sweep")
pscene.remove_binder("sweep_face")
initial_state = pscene.initialize_state(crob.home_pose)

In [26]:
from pkg.planning.pddlstream.plan_rnb import solve_in_pddlstream, play_pddl_plan

In [27]:
goal_pairs = [('box1', 'goal')]
res, plan, log_dict = solve_in_pddlstream(pscene, mplan, 'indy0', 'grip0', crob.home_pose, goal_pairs,
                                          TIMEOUT_MOTION=0.5, MAX_TIME=30, MAX_ITER=100, MAX_SKELETONS=10,
                                          GRASP_SAMPLE=30, STABLE_SAMPLE=50, SHOW_STATE=False, SEARCH_SAMPLE_RATIO=10,
                                          use_pybullet_gui=False, USE_MOVEIT_IK=True)

[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
Objects: {1L: 'goal', 2L: 'gripper', 3L: 'floor', 4L: 'wall', 5L: 'obstacle', 6L: 'box1'}
================ MAKE PROBLEM ======================
IK checkers: ['GraspChecker']
MP checkers: []
timeout motion : 0.5
Robot: 0
Movable: [6L]
Fixed: [1, 2, 3, 4, 5]
body 6 - surface 1
body 6 - surface 2
body 6 - surface 3
body 6 - surface 4
body 6 - surface 5
[INFO] Update stable set for box1 - goal
[INFO] Update stable set for box1 - floor
[INFO] Update grasp set for box1
Init: [('CanMove',), ('Conf', q0), ('AtConf', q0), ('HandEmpty',), ('Timer', 0L), ('Graspable', 6L), ('Pose', 6L, p0), ('AtPose', 6L, p0), ('Stackable', 6L, 1), ('Stackable', 6L, 2), ('Stackable', 6L, 3), ('Supported', 6L, p0, 3), ('Stackable', 6L, 4), ('Stackable', 6L, 5)]
Goal: ('and', ('AtConf', q0), ('On', 6L, 1L))
Streams: {TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test

In [28]:
play_pddl_plan(pscene, pscene.actor_dict["grip0"], initial_state=initial_state,
               body_names=log_dict['body_names'], plan=plan, SHOW_PERIOD=0.01)